<a href="https://colab.research.google.com/github/A-Mahla/serda-formation-python/blob/main/exercices/google_colab/Bases_du_Client_Python_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bases du Client Python OpenAI

## Introduction aux API OpenAI

### Qu'est-ce qu'une API ?

  - Une API (Interface de Programmation d'Applications) permet à des programmes d'interagir entre eux. Dans le cas d'OpenAI, elle permet aux développeurs d'envoyer des prompts (instructions) et de recevoir des réponses générées par les modèles GPT.



### Le client Python OpenAI

- Le package Python `openai` est un client officiel qui facilite l'accès aux API OpenAI depuis des applications Python. Il gère les détails de la communication réseau, la sérialisation des données, et les bonnes pratiques de sécurité.

## Installation et Configuration

### Installation du package

- Assurez-vous que Python est installé.
- Installez le package en utilisant pip :

```bash
pip install openai
```

In [ ]:
%pip install openai

### Configuration de la clé API

- La clé API est essentielle pour authentifier vos requêtes auprès de l'API OpenAI.
- Il est recommandé de ne jamais coder en dur votre clé API dans vos scripts. Utilisez plutôt des variables d'environnement :

```python
import os
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
```

In [3]:
# Pour les besoins de ce cours, nous allons démontrer comment utiliser la clé API.
# Cependant, il est crucial de ne jamais exposer publiquement votre clé API dans votre code ou sur des plateformes partagées.
# Dans un environnement de production, stockez toujours les clés API dans des variables d'environnement ou des services de gestion sécurisée des secrets.

openai_api_key = "votre_clef_api"  # Remplacez "votre_clé_api" par la clé API Openai.

## Réalisation d'un appel API simple

### 1. Création du client OpenAI

Avant de faire une requête, nous devons créer un client OpenAI en utilisant notre clé API. Cela permet d'authentifier nos appels API et de gérer la session de communication avec le service OpenAI.

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key=openai_api_key,
)

### 2. Envoyer une requête de complétion de texte

- Utilisation de la méthode `chat.completions.create()` du client pour demander des complétions de texte au modèle GPT.
- Paramètres de base :
  - `model`: spécifie le modèle à utiliser, par exemple "gpt-3.5-turbo-0125".
  - `messages`: Une liste de dictionnaires, chaque dictionnaire représentant un message de l'utilisateur ou du système. Pour conserver le contexte ou fournir des paires de questions-réponses afin de guider le modèle (approche de few-shot), on utilise les rôles user et assistant pour distinguer les questions des réponses.
  - `max_tokens`: le nombre maximum de tokens (mots ou morceaux de mots) que la réponse peut contenir.

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Expliquez ce qu'est une API et son utilité.",
        }
    ],
    model="gpt-3.5-turbo",
    max_tokens=100
)

print(completion.choices[0].message.content)

### 3. Exploration des paramètres avancés

- `temperature`: contrôle le degré de créativité et de déviation par rapport au prompt. Plus la température est élevée, plus les réponses sont uniques et moins prévisibles.
- `top_p`: méthode de sampling qui permet de contrôler la diversité de la réponse générée en limitant la sélection à un sous-ensemble de tokens possibles.

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Créez une courte histoire sur un voyage dans l'espace.",
        }
    ],
    model="gpt-3.5-turbo",
    max_tokens=150,
    temperature=0.7,
    top_p=1
)

print(completion.choices[0].message.content)

### Généreration en JSON

Une façon courante d'utiliser les complétions de conversation est de demander au modèle de toujours renvoyer un objet JSON qui correspond à votre cas d'utilisation, en spécifiant cela dans le message système. Bien que cela fonctionne dans certains cas, il arrive parfois que les modèles génèrent des sorties qui ne se traduisent pas en objets JSON valides.

Vous pouvez définir le format de réponse à { "type": "json_object" } pour activer le mode JSON. Lorsque le mode JSON est activé, le modèle est contraint de générer uniquement des chaînes qui se traduisent en un objet JSON valide.

**Notes importantes :**

- Lorsque vous utilisez le mode JSON, demandez toujours explicitement au modèle de produire du JSON via un message dans la conversation, par exemple via votre message système. Si vous n'incluez pas d'instruction explicite pour générer du JSON, le modèle peut générer un flux continu d'espaces blancs et la requête peut continuer indéfiniment jusqu'à atteindre la limite de tokens. Pour vous aider à ne pas oublier, l'API générera une erreur si la chaîne "JSON" n'apparaît pas quelque part dans le contexte.
- Le JSON dans le message que le modèle renvoie peut être partiel (c'est-à-dire tronqué) si la raison de fin est la longueur, ce qui indique que la génération a dépassé le nombre maximum de tokens ou que la conversation a dépassé la limite de tokens. Pour vous prémunir contre cela, vérifiez la raison de fin avant d'analyser la réponse.
- Le mode JSON ne garantit pas que la sortie corresponde à un schéma spécifique, seulement qu'elle est valide et s'analyse sans erreurs."


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  max_tokens=150,
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

### 4. Bonnes pratiques

#### 1. Gestion des erreurs

Il est important de gérer les exceptions et les erreurs réseau lors de l'utilisation de l'API.

In [ ]:
client = OpenAI(
    api_key="blablabla",
)

try:
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Racontez une blague sur les programmeurs.",
            }
        ],
        model="gpt-3.5-turbo",
        max_tokens=50
    )

    print(completion.choices[0].message.content)
except Exception as e:
    print(f"Une erreur est survenue: {e}")

#### 2. Optimisation des coûts

Les requêtes à l'API sont facturées en fonction du nombre de tokens. Il est donc judicieux d'optimiser vos prompts et de bien choisir le nombre de tokens pour contrôler vos dépenses.

## Exercice Pratique

- Exercice 1: Premier appel API

  - Écrivez un script qui demande à l'utilisateur un thème et utilise GPT pour générer ce qu'il vous plaira :)

In [ ]:
# Your Code ...





- Exercice 2: Exploration des paramètres

  - Expérimentez avec les paramètres temperature et max_tokens pour observer les variations dans les réponses générées.

In [ ]:
# Your Code ...





- Exercice 3: Compréhension du contexte
  - Le système doit maintenir un contexte de conversation pour que les réponses aux questions suivantes puissent prendre en compte les informations fournies précédemment par l'utilisateur.
  - Par exemple, si l'utilisateur demande "Quelles sont les meilleures destinations en Europe ?", et suit avec "Quel est le meilleur moment pour y aller ?", le système devrait comprendre que la deuxième question se réfère toujours aux destinations européennes.

In [ ]:
# Your Code ...



